<a href="https://colab.research.google.com/github/cbologa/public_databases/blob/main/python/ChEMBL_get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title
%load_ext google.colab.data_table

Install and import required packages

In [2]:
#@title
!pip install -q chembl-downloader
import chembl_downloader

Download last version of ChEMBL (it takes 2-20 min)

In [3]:
#@title
path = chembl_downloader.download_extract_sqlite(version='31')

Define SQL query

In [4]:
#@title
sql = """
select
  target_dictionary.*,
  assays.*,
  activities.*,
  compound_structures.standard_inchi_key,
  compound_structures.canonical_smiles,
  molecule_dictionary.*,
  docs.*
from
  target_dictionary
  left join assays on target_dictionary.tid=assays.tid
  left join docs on assays.doc_id=docs.doc_id
  left join activities on assays.assay_id=activities.assay_id
  left join molecule_dictionary on activities.molregno=molecule_dictionary.molregno
  left join compound_structures on molecule_dictionary.molregno=compound_structures.molregno
where 
  target_dictionary.organism like '%equine encephalitis virus'
group by
  target_dictionary.tid,
  assays.assay_id,
  activities.activity_id,
  compound_structures.standard_inchi_key,
  compound_structures.canonical_smiles,
  molecule_dictionary.molregno,
  docs.doc_id
order by
  docs.year,
  docs.doi
"""

Query database and save to file

In [7]:
#@title
df = chembl_downloader.query(sql)

#Save to file
df.to_csv("eev.tsv", sep='\t', index=False)

#Show results
df

,tid,target_type,pref_name,tax_id,organism,chembl_id,species_group_flag,assay_id,doc_id,description,...,pubmed_id,doi,chembl_id,title,doc_type,authors,abstract,patent_id,ridx,src_id
0,50637,ORGANISM,Venezuelan equine encephalitis virus,11036,Venezuelan equine encephalitis virus,CHEMBL613726,0,1359874,83427,Antiviral activity against Venezuelan equine e...,...,191610,10.1021/jm00213a011,CHEMBL3272338,Bis-basic-substituted polycyclic aromatic comp...,PUBLICATION,"Albrecht WL, Fleming RW, Horgan SW, Mayer GD.","A series of bisalkamine esters, bis-basic ethe...",None,CLD0,1
1,50637,ORGANISM,Venezuelan equine encephalitis virus,11036,Venezuelan equine encephalitis virus,CHEMBL613726,0,1359874,83427,Antiviral activity against Venezuelan equine e...,...,191610,10.1021/jm00213a011,CHEMBL3272338,Bis-basic-substituted polycyclic aromatic comp...,PUBLICATION,"Albrecht WL, Fleming RW, Horgan SW, Mayer GD.","A series of bisalkamine esters, bis-basic ethe...",None,CLD0,1
2,50637,ORGANISM,Venezuelan equine encephalitis virus,11036,Venezuelan equine encephalitis virus,CHEMBL613726,0,215195,8662,In vitro antiviral activity was tested against...,...,2993613,10.1021/jm00147a013,CHEMBL1123027,Synthesis and biological activity of 5-thiobre...,PUBLICATION,"Wood SG, Upadhya KG, Dalley NK, McKernan PA, C...",A number of 5-substituted imidazole-4-carboxam...,None,CLD0,1
3,50637,ORGANISM,Venezuelan equine encephalitis virus,11036,Venezuelan equine encephalitis virus,CHEMBL613726,0,215195,8662,In vitro antiviral activity was tested against...,...,2993613,10.1021/jm00147a013,CHEMBL1123027,Synthesis and biological activity of 5-thiobre...,PUBLICATION,"Wood SG, Upadhya KG, Dalley NK, McKernan PA, C...",A number of 5-substituted imidazole-4-carboxam...,None,CLD0,1
4,50637,ORGANISM,Venezuelan equine encephalitis virus,11036,Venezuelan equine encephalitis virus,CHEMBL613726,0,215196,8662,In vitro antiviral activity was tested against...,...,2993613,10.1021/jm00147a013,CHEMBL1123027,Synthesis and biological activity of 5-thiobre...,PUBLICATION,"Wood SG, Upadhya KG, Dalley NK, McKernan PA, C...",A number of 5-substituted imidazole-4-carboxam...,None,CLD0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,50637,ORGANISM,Venezuelan equine encephalitis virus,11036,Venezuelan equine encephalitis virus,CHEMBL613726,0,2126265,121014,Antiviral activity against Venezuelan equine e...,...,33835811,10.1021/acs.jmedchem.0c02183,CHEMBL4834424,Targeting Chikungunya Virus Replication by Ben...,PUBLICATION,"Ahmed SK, Haese NN, Cowan JT, Pathak V, Moukha...","A benzo[6]annulene, 4-(<i>tert</i>-butyl)-<i>N...",None,33835811,1
292,50637,ORGANISM,Venezuelan equine encephalitis virus,11036,Venezuelan equine encephalitis virus,CHEMBL613726,0,2126265,121014,Antiviral activity against Venezuelan equine e...,...,33835811,10.1021/acs.jmedchem.0c02183,CHEMBL4834424,Targeting Chikungunya Virus Replication by Ben...,PUBLICATION,"Ahmed SK, Haese NN, Cowan JT, Pathak V, Moukha...","A benzo[6]annulene, 4-(<i>tert</i>-butyl)-<i>N...",None,33835811,1
293,50637,ORGANISM,Venezuelan equine encephalitis virus,11036,Venezuelan equine encephalitis virus,CHEMBL613726,0,2126279,121014,Antiviral activity against Venezuelan equine e...,...,33835811,10.1021/acs.jmedchem.0c02183,CHEMBL4834424,Targeting Chikungunya Virus Replication by Ben...,PUBLICATION,"Ahmed SK, Haese NN, Cowan JT, Pathak V, Moukha...","A benzo[6]annulene, 4-(<i>tert</i>-butyl)-<i>N...",None,33835811,1
294,50637,ORGANISM,Venezuelan equine encephalitis virus,11036,Venezuelan equine encephalitis virus,CHEMBL613726,0,2126279,121014,Antiviral activity against Venezuelan equine e...,...,33835811,10.1021/acs.jmedchem.0c02183,CHEMBL4834424,Targeting Chikungunya Virus Replication by Ben...,PUBLICATION,"Ahmed SK, Haese NN, Cowan JT, Pathak V, Moukha...","A benzo[6]annulene, 4-(<i>tert</i>-butyl)-<i>N...",None,33835811,1
